In [ ]:
pip install beautifulsoup4

In [ ]:
pip install lxml

In [ ]:
pip install selenium

In [ ]:
pip install webdriver_manager

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
import csv
import os
os.chdir(r"C:\Users\sharm\OneDrive\Desktop\AUD")

In [3]:
import pandas as pd
import re
import random
from datetime import datetime
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import requests
import os
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [4]:
headers_list = [
    # Firefox 77 Mac
    {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 92.0 Win10
    {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 91.0 Win10
    {
    "Connection": "keep-alive",
    "DNT": "1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
    },
    # Firefox 90.0 Win10
    {
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-User": "?1",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9"
    }
]
headers = random.choice(headers_list)
chrome_options = Options()
chrome_options.add_argument("user-agent=%s" % headers) 
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox') # operate at the highest authority
chrome_options.add_argument('--disable-dev-shm-usage') #increase the RAM of chrome to load the page
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = chrome_options)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.58M/6.58M [00:00<00:00, 24.8MB/s]


In [5]:
url="https://tippecanoe.craigslist.org/search/lafayette-in/moa?lat=40.44&lon=-86.877&sort=date&srchType=T&hasPic=1&search_distance=250&min_price=30&max_price="
driver.get(url)

In [6]:
html=driver.page_source

In [7]:
soup = bs(html, 'html.parser')

In [8]:
data={'id':[],'title':[],'description':[],'link':[],'lat':[], 'long':[],'price':[]}

In [9]:
end=False

while end == False:
    rel=soup.find_all('h3',class_='result-heading')
    for i in range(len(rel)):

        check=rel[i].a['href']
        
        headers = random.choice(headers_list)
        r = requests.Session()
        retry = Retry(connect=5, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        r.mount('http://', adapter)
        r.mount('https://', adapter)
        r.headers = headers
        html1 = r.get(check).text
        
        soup1 = bs(html1, 'html.parser')
        
        # Change to relevant parameters from here
        
        temp1 = soup1.find_all('div',class_='postinginfos')
        
        try:
            for i in temp1[0].find_all('p',class_='postinginfo'):
                posting_id= int(re.findall('[0-9]+', i.get_text())[0])
                break
        except:
            continue
            
        data['id'].append(posting_id)
        
        # title of listing
        
        temp2 = soup1.find_all('span',class_='postingtitletext')
        
        for i in temp2:
            posting_title=i.get_text()
        
        data['title'].append(posting_title.strip())
        
        #text of listing
        
        temp3 = soup1.find_all('section',id="postingbody")
        
        for i in temp3:
            posting_text=i.get_text()
        
        data['description'].append(posting_text.strip())
        
        
        data['link'].append(check)

        
        try:
            temp123 = soup1.find('div',{"id":"map"})['data-latitude']
        except:
            continue
        data['lat'].append(temp123)

        
        try:
            temp123 = soup1.find('div',{"id":"map"})['data-longitude']
        except:
            continue
        data['long'].append(temp123)
        
        try:
            price = soup1.find('span',{"class":"price"})
        except:
            continue
        data['price'].append(price.text)
        
        #images_links and download
        
#         links=[]
        
#         temp4 = soup1.find_all('div',id='thumbs')
        
#         try:
#             for i in temp4[0].find_all('a',href=True):
#                  links.append(i['href'])
#         except:
#                 temp5 = soup1.find_all('div',class_='swipe')        
#                 try:
#                     for i in temp5[0].find_all('img'):
#                             links.append(i['src'])
#                 except:
#                     pass
        
#         if len(links) != 0: 
            
#             for i in range(len(links)):
                
#                 img_data = requests.get(links[i]).content
                
#                 #try:
#                 #    os.makedirs(r'C:\Users\sanka\Desktop\images\{}'.format(posting_id))
#                 #except:
#                 #    pass
#                 #with open(r'C:\Users\sanka\Desktop\images\{}\img_{}.jpg'.format(posting_id,i+1), 'wb') as handler:
#                 #    handler.write(img_data)
#                 with open(r'C:\Manmeet Walia\MSBA\AUD\Final Project\Images\{}_img_{}.jpg'.format(posting_id,i+1), 'wb') as handler:
#                     handler.write(img_data)
    
    
    buttons=soup.find_all('span',class_='buttons')
    
    for i in buttons[0].find_all('a',class_='button next'):
        button_link = i['href']
    
    if len(button_link) != 0:
        
        link_text=soup.find_all('h1',class_='cattitle')
        
        for i in link_text[0].find_all('a',href=True):
            pre_link = i['href']
        
        link_split=pre_link.replace('//','/').split('/')
        
        next_page='https://'+ link_split[1] + button_link
        
        headers = random.choice(headers_list)
        r = requests.Session()
        retry = Retry(connect=5, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        r.mount('http://', adapter)
        r.mount('https://', adapter)
        r.headers = headers
        html2 = r.get(next_page).text
        
        soup = bs(html2, 'html.parser')
    
    else:
        end = True


output=pd.DataFrame(data)

output.to_csv('abcd.csv',index=False)        

In [10]:
df=pd.read_csv("abcd.csv")

In [11]:
df.head()

,id,title,description,link,lat,long,price
0,7562104793,brand new factory sealed Verizon Blackberry 87...,QR Code Link to This Post\n\n\nVerizon Blackbe...,https://chicago.craigslist.org/nwc/mob/d/brand...,41.979400,-88.20630,$100
1,7562104702,brand new factory sealed Verizon Blackberry 71...,QR Code Link to This Post\n\n\nVerizon Blackbe...,https://chicago.craigslist.org/nwc/mob/d/bartl...,41.979400,-88.20630,$100
2,7565695339,IPHONE SCREEN REPAIR SPECIAL - $50 (Saint Louis),QR Code Link to This Post\n\n\nIPHONE 6S $40\n...,https://stlouis.craigslist.org/mod/d/saint-lou...,38.702300,-90.36440,$50
3,7565667427,IPHONE BROKEN GLASS REPAIR HOLIDAY SALE - $50 ...,QR Code Link to This Post\n\n\nIPHONE BROKEN G...,https://detroit.craigslist.org/wyn/mob/d/south...,42.501924,-83.24301,$50
4,7565673909,Apple IPhone 13 256GB - $500 (Columbus),QR Code Link to This Post\n\n\niPhone is brand...,https://columbus.craigslist.org/mob/d/columbus...,39.923000,-82.86640,$500


In [12]:
dfnormal=df.copy()

In [13]:
dfnormal["lat"]=(df["lat"]-df["lat"].mean())/df["lat"].std()

In [14]:
dfnormal["long"]=(df["long"]-df["long"].mean())/df["long"].std()

In [15]:
df['price'] = df['price'].str.replace(',', '')
df['price'] =df['price'].str.replace('$', '')
df['price'] = df['price'].astype(int)

C:\Users\sharm\AppData\Local\Temp/ipykernel_34864/3915003914.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['price'] =df['price'].str.replace('$', '')


In [16]:
dfnormal["price"]=(df["price"]-df["price"].mean())/df["price"].std()

In [17]:
dfnormal.head()

,id,title,description,link,lat,long,price
0,7562104793,brand new factory sealed Verizon Blackberry 87...,QR Code Link to This Post\n\n\nVerizon Blackbe...,https://chicago.craigslist.org/nwc/mob/d/brand...,0.503840,-0.769870,-0.367496
1,7562104702,brand new factory sealed Verizon Blackberry 71...,QR Code Link to This Post\n\n\nVerizon Blackbe...,https://chicago.craigslist.org/nwc/mob/d/bartl...,0.503840,-0.769870,-0.367496
2,7565695339,IPHONE SCREEN REPAIR SPECIAL - $50 (Saint Louis),QR Code Link to This Post\n\n\nIPHONE 6S $40\n...,https://stlouis.craigslist.org/mod/d/saint-lou...,-1.569450,-1.663623,-0.456576
3,7565667427,IPHONE BROKEN GLASS REPAIR HOLIDAY SALE - $50 ...,QR Code Link to This Post\n\n\nIPHONE BROKEN G...,https://detroit.craigslist.org/wyn/mob/d/south...,0.834420,1.285621,-0.456576
4,7565673909,Apple IPhone 13 256GB - $500 (Columbus),QR Code Link to This Post\n\n\niPhone is brand...,https://columbus.craigslist.org/mob/d/columbus...,-0.797162,1.441590,0.345144


In [19]:
dfnormal.to_csv(r'abcdNormalized.csv',index=False)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
plt.xticks(rotation = 90)
plt.hist(df["title"])

<!-- 521 rows
No null values -->

In [ ]:
lst=[]
for title in df["title"]:
    lst.append(title.lower())

In [ ]:
iPhone=0
Nokia=0
Samsung=0
Pixel=0
Motorola=0
OnePlus=0
alcatel=0
Huawei=0
LG=0
Others=0
for title in lst:
    if re.search("iphone",title.lower()):
        iPhone+=1
    elif re.search("nokia",title):
        Nokia+=1
    elif re.search("samsung",title):
        Samsung+=1
    elif re.search("pixel",title):
        Pixel+=1
    elif re.search("moto",title):
        Motorola+=1
    elif re.search("oneplus",title):
        OnePlus+=1
    elif re.search("alcatel",title):
        alcatel+=1
    elif re.search("huawei",title):
        Huawei+=1
    elif re.search("lg",title):
        LG+=1
    else:
        Others+=1

    

In [ ]:
iPhone

In [ ]:
Others

In [ ]:
Samsung

In [ ]:
Pixel

In [ ]:
Motorola

In [ ]:
!pip install nltk

In [4]:
import nltk

In [5]:
from nltk.stem import WordNetLemmatizer

In [20]:
df = dfnormal

In [182]:
df.head()

,id,title,description,link,lat,long,price
0,7562104793,brand new factory sealed Verizon Blackberry 87...,QR Code Link to This Post\n\n\nVerizon Blackbe...,https://chicago.craigslist.org/nwc/mob/d/brand...,0.503840,-0.769870,-0.367496
1,7562104702,brand new factory sealed Verizon Blackberry 71...,QR Code Link to This Post\n\n\nVerizon Blackbe...,https://chicago.craigslist.org/nwc/mob/d/bartl...,0.503840,-0.769870,-0.367496
2,7565695339,IPHONE SCREEN REPAIR SPECIAL - $50 (Saint Louis),QR Code Link to This Post\n\n\nIPHONE 6S $40\n...,https://stlouis.craigslist.org/mod/d/saint-lou...,-1.569450,-1.663623,-0.456576
3,7565667427,IPHONE BROKEN GLASS REPAIR HOLIDAY SALE - $50 ...,QR Code Link to This Post\n\n\nIPHONE BROKEN G...,https://detroit.craigslist.org/wyn/mob/d/south...,0.834420,1.285621,-0.456576
4,7565673909,Apple IPhone 13 256GB - $500 (Columbus),QR Code Link to This Post\n\n\niPhone is brand...,https://columbus.craigslist.org/mob/d/columbus...,-0.797162,1.441590,0.345144


In [183]:
titles = df.title.values.tolist() 

In [184]:
id_ = df.id.values.tolist() 

### 1: Tokenizing 

In [185]:
tokenized_reviews = []

In [186]:
for title in titles:
    tokenized_reviews.append(nltk.word_tokenize(title))

In [187]:
tokenized_reviews[:5]   # It is a list of lists

[['brand',
  'new',
  'factory',
  'sealed',
  'Verizon',
  'Blackberry',
  '8703e',
  '-',
  '$',
  '100',
  '(',
  'Bartlett',
  ')'],
 ['brand',
  'new',
  'factory',
  'sealed',
  'Verizon',
  'Blackberry',
  '7130e',
  '-',
  '$',
  '100',
  '(',
  'Bartlett',
  ')'],
 ['IPHONE',
  'SCREEN',
  'REPAIR',
  'SPECIAL',
  '-',
  '$',
  '50',
  '(',
  'Saint',
  'Louis',
  ')'],
 ['IPHONE',
  'BROKEN',
  'GLASS',
  'REPAIR',
  'HOLIDAY',
  'SALE',
  '-',
  '$',
  '50',
  '(',
  'SOUTHFIELD',
  ')'],
 ['Apple', 'IPhone', '13', '256GB', '-', '$', '500', '(', 'Columbus', ')']]

In [188]:
tokenized_reviews = [[string.lower() for string in sublist] for sublist in tokenized_reviews]

In [189]:
tokenized_reviews

[['brand',
  'new',
  'factory',
  'sealed',
  'verizon',
  'blackberry',
  '8703e',
  '-',
  '$',
  '100',
  '(',
  'bartlett',
  ')'],
 ['brand',
  'new',
  'factory',
  'sealed',
  'verizon',
  'blackberry',
  '7130e',
  '-',
  '$',
  '100',
  '(',
  'bartlett',
  ')'],
 ['iphone',
  'screen',
  'repair',
  'special',
  '-',
  '$',
  '50',
  '(',
  'saint',
  'louis',
  ')'],
 ['iphone',
  'broken',
  'glass',
  'repair',
  'holiday',
  'sale',
  '-',
  '$',
  '50',
  '(',
  'southfield',
  ')'],
 ['apple', 'iphone', '13', '256gb', '-', '$', '500', '(', 'columbus', ')'],
 ['apple',
  'iphone',
  '13',
  'pro',
  'max',
  't-mobile',
  '-',
  '$',
  '700',
  '(',
  'columbus',
  ')'],
 ['iphone',
  'broken',
  'glass',
  'repair',
  'holiday',
  'sale',
  '-',
  '$',
  '50',
  '(',
  'southfield',
  ')'],
 ['iphone',
  'broken',
  'glass',
  'repair',
  'holiday',
  'sale',
  '-',
  '$',
  '50',
  '(',
  'southfield',
  ')'],
 ['factory', 'unlocked', 'iphone', '7', 'black', '.', '-',

In [190]:
wnl = WordNetLemmatizer()

In [191]:
lemmatized_reviews = [[wnl.lemmatize(word) for word in sublist] for sublist in tokenized_reviews]

### 3: Removing stop-words and punctuations

In [192]:
#removing stop words
from nltk.corpus import stopwords

In [193]:
import string

In [194]:
stop = set(stopwords.words('english') + list(string.punctuation))

In [195]:
stop_words_removed = [[token for token in sublist if not token in
stop if (token.isalpha() or token.isdigit())] for sublist in lemmatized_reviews]

In [196]:
stop_words_removed

[['brand',
  'new',
  'factory',
  'sealed',
  'verizon',
  'blackberry',
  '100',
  'bartlett'],
 ['brand',
  'new',
  'factory',
  'sealed',
  'verizon',
  'blackberry',
  '100',
  'bartlett'],
 ['iphone', 'screen', 'repair', 'special', '50', 'saint', 'louis'],
 ['iphone',
  'broken',
  'glass',
  'repair',
  'holiday',
  'sale',
  '50',
  'southfield'],
 ['apple', 'iphone', '13', '500', 'columbus'],
 ['apple', 'iphone', '13', 'pro', 'max', '700', 'columbus'],
 ['iphone',
  'broken',
  'glass',
  'repair',
  'holiday',
  'sale',
  '50',
  'southfield'],
 ['iphone',
  'broken',
  'glass',
  'repair',
  'holiday',
  'sale',
  '50',
  'southfield'],
 ['factory', 'unlocked', 'iphone', '7', 'black', '159'],
 ['factory', 'unlocked', 'iphone', '7', 'silver', '159'],
 ['iphone', 'xr', 'refurbished', '200', 'clef'],
 ['iphone',
  'broken',
  'glass',
  'repair',
  'holiday',
  'sale',
  '50',
  'southfield'],
 ['iphone',
  'broken',
  'glass',
  'repair',
  'holiday',
  'sale',
  '50',
  'sou

In [197]:
combined_words = []

In [198]:
for word in stop_words_removed:
    combined_words.append(" ".join(word))

In [199]:
from sklearn.feature_extraction.text import CountVectorizer

In [200]:
vectorizer1 = CountVectorizer(ngram_range=(1,4),min_df=5)
vectorizer1.fit(combined_words)

CountVectorizer(min_df=5, ngram_range=(1, 4))

In [201]:
v1 = vectorizer1.transform(combined_words)
print(v1.toarray())


[[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [202]:
cols = vectorizer1.get_feature_names_out() 

In [203]:
df1 = pd.DataFrame(v1.toarray(), columns=cols)

In [204]:
df1['id'] = df['id']

In [205]:
df1.head()

,10,100,105,11,11 pro,11 pro max,11 unlocked,12,12 mini,12 pro,...,wi,wifi,wireless,work,xr,ypsilanti,zte,zte blade,zte blade prime,id
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7562104793
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7562104702
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7565695339
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7565667427
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7565673909


In [206]:
cols = list(df1)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('id')))
cols
#Out[27]:
#['Mid', 'Net', 'Upper', 'Lower', 'Zsore']
#In [28]:
# use ix to reorder
df1 = df1.loc[:, cols]
df1

,id,10,100,105,11,11 pro,11 pro max,11 unlocked,12,12 mini,...,white,wi,wifi,wireless,work,xr,ypsilanti,zte,zte blade,zte blade prime
0,7562104793,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7562104702,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7565695339,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7565667427,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7565673909,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,7549089164,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1198,7549070916,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1199,7549070655,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1200,7543638335,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [207]:
df1['lat'] = df['lat']

In [208]:
df1['long'] = df['long']

In [209]:
df1['price'] = df['price']

In [210]:
df1.to_csv('bagofwords.csv')

## Using TF-IDF matrix

In [211]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [212]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer2 = TfidfVectorizer(ngram_range=(1,5),min_df=5)
vectorizer2.fit(combined_words)
#print(vectorizer2.vocabulary_)
v2 = vectorizer2.transform(combined_words)
#print(v2.toarray())

In [213]:
cols = vectorizer2.get_feature_names_out() 

In [214]:
df2 = pd.DataFrame(v2.toarray(), columns=cols)

In [215]:
df2.head()

,10,100,105,11,11 pro,11 pro max,11 unlocked,12,12 mini,12 pro,...,white,wi,wifi,wireless,work,xr,ypsilanti,zte,zte blade,zte blade prime
0,0.0,0.307861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.307861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [216]:
df2['id'] = df['id']

In [217]:
cols = list(df2)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('id')))
cols
#Out[27]:
#['Mid', 'Net', 'Upper', 'Lower', 'Zsore']
#In [28]:
# use ix to reorder
df2 = df2.loc[:, cols]
df2

,id,10,100,105,11,11 pro,11 pro max,11 unlocked,12,12 mini,...,white,wi,wifi,wireless,work,xr,ypsilanti,zte,zte blade,zte blade prime
0,7562104793,0.0,0.307861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7562104702,0.0,0.307861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7565695339,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7565667427,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7565673909,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,7549089164,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1198,7549070916,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1199,7549070655,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1200,7543638335,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [218]:
df2['lat'] = df['lat']

In [219]:
df2['long'] = df['long']

In [220]:
df2['price'] = df['price']

In [221]:
df2['title'] = df['title']

In [21]:
df = df2

NameError: name 'df2' is not defined

In [681]:

df = df.set_index('id')
df1 = df.drop(columns = ['Unnamed: 0', 'title'])

In [682]:
def knn(df, query_index):
    import decimal as D
    from sklearn.neighbors import NearestNeighbors
    model_knn = NearestNeighbors(metric = "cosine", algorithm = "brute", n_neighbors = 6, n_jobs =- 1)
    model_knn.fit(df)
    distances, indices = model_knn.kneighbors(df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
    #print("Indices:", indices, "\n\nDistances:", distances.astype(D.Decimal))
    print("Recommendations for {0} are:\n".format(df.index[query_index]))
    for i in range(0,len(distances.flatten())):
        if df.index[indices.flatten()[i]] == df.index[query_index]:
            pass
        else:
            print("{0} with distance of {1}".format(df.index[indices.flatten()[i]], distances.flatten()[i]))

In [688]:
knn(df1, 5)

Recommendations for 7565676662 are:

7553032191 with distance of 0.07084702834598022
7565673909 with distance of 0.11034739785470493
7564549967 with distance of 0.11034739785470493
7541021311 with distance of 0.11437533035598868
7564960178 with distance of 0.12344359645746372


/Users/mandeepsinghrahi/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [689]:
def knn2(df, query_index):
    import decimal as D
    from sklearn.neighbors import NearestNeighbors
    model_knn = NearestNeighbors(metric = "cosine", algorithm = "brute", n_neighbors = 6, n_jobs =- 1)
    model_knn.fit(df)
    distances, indices = model_knn.kneighbors(df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
    #print("Indices:", indices, "\n\nDistances:", distances.astype(D.Decimal))
    return pd.DataFrame(data={'query': [df.index[query_index]], 'knnrec1': [df.index[indices.flatten()[0]]],
                              'knnrec2': [df.index[indices.flatten()[1]]], 'knnrec3': [df.index[indices.flatten()[2]]], 
                              'knnrec4': [df.index[indices.flatten()[3]]], 'knnrec5': [df.index[indices.flatten()[4]]],
                              'knnrec6': [df.index[indices.flatten()[5]]], 'knndist1': [distances.flatten()[0]], 
                              'knndist2': [distances.flatten()[1]], 'knndist3': [distances.flatten()[2]], 
                              'knndist4': [distances.flatten()[3]], 'knndist5': [distances.flatten()[4]],
                              'knndist6': [distances.flatten()[5]]})

In [690]:
results = pd.DataFrame(columns=['query','knnrec1','knnrec2','knnrec3','knnrec4','knnrec5','knnrec6',
                                'knndist1','knndist2','knndist3','knndist4','knndist5','knndist6'])     

for i in range(len(df1)):
    results = pd.concat([results, pd.DataFrame(knn2(df1, i))])

/Users/mandeepsinghrahi/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/mandeepsinghrahi/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/mandeepsinghrahi/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/mandeepsinghrahi/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Users/mandeepsinghrahi/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with featur

In [ ]:
pip install datasketch

In [ ]:
pip install re

In [691]:
from datasketch import MinHash, MinHashLSHForest
import re

In [692]:
df2 = df[['title']].reset_index()

In [693]:
def LSH(df, query_index):
    
    def get_forest(data):
    
        minhash = []
    
        for text in data['title']:
            tokens = text.split()
            m = MinHash(num_perm = 512)
            for s in tokens:
                m.update(s.encode('utf8'))
            minhash.append(m)
        
        forest = MinHashLSHForest(num_perm = 512)
    
        for i, m in enumerate(minhash):
            forest.add(i,m)
        
        forest.index()
    
        return forest
    
    forest = get_forest(db)
    
    tokens = df.loc[query_index, 'title'].split()
    m = MinHash(num_perm = 512)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    indices = np.array(forest.query(m, 6))
    #print("Indices:", indices, "\n")
    print("Recommendations for {0} are:\n".format(df['id'][df.index[query_index]]))
    for i in range(0, len(indices.flatten())):
        if df.index[indices.flatten()[i]] == df.index[query_index]:
            pass
        else:
            print("{0}".format(df['id'][df.index[indices.flatten()[i]]]))

In [694]:
LSH(df2, 22)

Recommendations for 7565646856 are:

7560612573
7564625611
7565560931
7554408552
7558622539
7564359185


In [695]:
def LSH2(df, query_index):
    
    def get_forest(data):
    
        minhash = []
    
        for text in data['title']:
            tokens = text.split()
            m = MinHash(num_perm = 128)
            for s in tokens:
                m.update(s.encode('utf8'))
            minhash.append(m)
        
        forest = MinHashLSHForest(num_perm = 128)
    
        for i, m in enumerate(minhash):
            forest.add(i,m)
        
        forest.index()
    
        return forest
    
    forest = get_forest(db)
    
    tokens = df.loc[query_index, 'title'].split()
    m = MinHash(num_perm = 128)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    indices = np.array(forest.query(m, 6))
    #print("Indices:", indices, "\n")
    
    x = {'query': [df['id'][df.index[query_index]]]}
    for i in range(0, len(indices.flatten())):
        if np.isnan([df['id'][df.index[indices.flatten()[i]]]]):
            x.update({'lshrec'+str(i+1): 0})
        else:
            x.update({'lshrec'+str(i+1): [df['id'][df.index[indices.flatten()[i]]]]})

    return pd.DataFrame(data=x)

In [697]:
results2 = pd.DataFrame(columns=['query','lshrec1','lshrec2','lshrec3','lshrec4','lshrec5','lshrec6'])

for i in range(0, len(df2)):
    results2 = pd.concat([results2, pd.DataFrame(LSH2(df2, i))])

In [700]:
results3 = results.join(results2.set_index('query'), on='query', how='left')
results3.to_csv('results.csv')

/Users/mandeepsinghrahi/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
